In [54]:
import os
import random
import numpy as np
import sys
import math
np.set_printoptions(threshold=sys.maxsize)

In [55]:
trainPath = "./train"
wordtag = {}
tags = {}
word_count = {}
word_UNK = {}
unitag = {}
bitag = {}
threshold = 5
transition_prob = {}
emission_prob = {}
txt = ""

In [56]:
for f in os.listdir(trainPath):
    fp = open(trainPath+"/"+f,"r+")
    txt = txt + fp.read().lower()

In [57]:
# txt = "	All/abn framing/nn in/in Hotei/np is/bez one-inch/jj mahogany/nn which/wdt ,/, in/in the/at dressed/vbn state/nn you/ppss buy/vb it/ppo ,/, is/bez about/rb the/at 13/16-inch/jj thickness/nn specified/vbn in/in the/at drawings/nns ./. Therefore/rb ,/, the/at lumber/nn is/bez bought/vbn in/in planks/nns and/cc ripped/vbn to/in size/nn for/in battens/nns ,/, etc./rb ,/, on/in a/at table/nn saw/nn ./. Besides/rb flathead/nn bronze/nn screws/nns ,/, silicon/nn bronze/nn Stronghold/nn-tl nails/nns (/( made/vbn by/in Independent/jj-tl Nail/nn-tl &/cc-tl Packing/nn-tl Co./nn-tl ,/, Bridgewater/np ,/, Mass./np )/) are/ber used/vbn extensively/rb in/in assembly/nn and/cc Weldwood/np resorcinol/nn glue/nn is/bez used/vbn in/in all/abn the/at joints/nns ./."

In [77]:
wordList = txt.replace("\n\n\t", " <end>/<end> <start>/<start> ").replace("\t","").replace("\n"," ").split()

In [81]:
wordList = [word for word in wordList if len(wordList)!= 0]

In [83]:
for i in range(1,len(wordList)):
    splitTag = wordList[i-1].split("/")
    if len(splitTag) > 2:
        wt = [None] * 2
        wt[0] = ("/".join(splitTag[:-1]))
        wt[1] = (splitTag[-1])
    else:
        wt = splitTag
    wtTuple = (wt[0],wt[1])
    if wtTuple in wordtag:
        wordtag[wtTuple] += 1
    else:
        wordtag[wtTuple] = 1
    if wt[0] in word_count:
        word_count[wt[0]] += 1
    else:
        word_count[wt[0]] = 1
    if wt[1] in unitag:
        unitag[wt[1]] += 1
    else:
        unitag[wt[1]] = 1
    tag2 =  wordList[i].split("/")
    if len(tag2) > 2:
        t = [None] * 2
        t[0] = "/".join(tag2[:-1])
        t2 = tag2[-1]
    else:
        t2 = tag2[1]
    if (wt[1],t2) in bitag:
        bitag[(wt[1],t2)] += 1
    else:
        bitag[(wt[1],t2)] = 1
    if (wt[1]) in tags:
        tags[wt[1]] += 1
    else:
        tags[wt[1]] = 1

In [84]:
word_UNK["UNK"] = 0
for k,v in word_count.items():
    if v <= threshold:
        word_UNK['UNK'] += v
    else:
        word_UNK[k] = v

In [85]:
for k,v in transition_prob.items():
    if 'vb' in k[1] and k[0] == 'cs':
        print(k[0] + " " + k[1])

In [86]:
for k,v in bitag.items():
    transition_prob[k] = float(bitag[k])/unitag[k[0]]

In [87]:
for k,v in wordtag.items():
    emission_prob[k] = float(v)/unitag[k[1]]

In [88]:
def draw_random_tag(tag,n):
    nextTagList = [(k,v) for k,v in transition_prob.items() if k[0] == tag]
    nextTagList.sort(key=lambda x: x[1],reverse=True)
    if(len(nextTagList) >= 3):
        return random.choice(nextTagList[:3])
    else:
        random.choice(nextTagList)

In [89]:
def draw_random_word(word,n):
    nextWordList = [(k,v) for k,v in emission_prob.items() if k[1] == word]
    nextWordList.sort(key=lambda x: x[1],reverse=True)
    if len(nextWordList) >= 10:
        return random.choice(nextWordList[:10])
    else:
        return random.choice(nextWordList) 

In [90]:
def generate_sentence():
    text = ""
    sentenceProb = 0
    cur_word = "<start>"
    cur_tag = "<start>"
    while True:
        next_tag_tuple = draw_random_tag(cur_tag, 1)
        next_tag = next_tag_tuple[0][1]

        next_word_tuple = draw_random_word(next_tag,1)
        cur_tag = next_tag_tuple[0][1]
        cur_word = next_word_tuple[0][0]
        sentenceProb = next_tag_tuple[1] * next_word_tuple[1]
        if cur_word == "<end>":
            break
        text = text + " " + cur_word
        
        
    return text[1:],sentenceProb

In [91]:
generate_sentence()

("`` time into own world by time . a such hands into an members : an things : either state from th' high on year : but no children ; every world : a world [ plus old years . ! th' state [ either american into every american children : no long children ] a high men : at af for such day [ into work to work ] yet ever' great world : the hands : an day ! . ?",
 0.21402086320264993)

In [92]:
def get_transmission_prob(tag1, tag2):
    return transition_prob.get((tag1,tag2),0)
def get_emission_prob(tag,word):
    return emission_prob.get((word,tag),0)

In [93]:
def viterbi(tList,tagsList): 
    N = len(tags)
    T = len(tList)
    viterbi = np.zeros((N,T))
    backpointer = np.zeros((N,T),dtype=int)
    for s in range(N): 
        viterbi[s][0] = get_transmission_prob('<start>',tagsList[s]) * get_emission_prob(tagsList[s],tList[0]) 
        backpointer[s][0] = 0
    for t in range(1,T):
        for s in range(N):
            viterbi[s][t] = max(viterbi[s_prime][t-1] * get_transmission_prob(tagsList[s_prime],tagsList[s]) * get_emission_prob(tagsList[s],tList[t]) 
                            for s_prime in range(N))
            backpointer[s][t] = np.argmax([viterbi[s_prime][t-1] * get_transmission_prob(tagsList[s_prime],tagsList[s]) * get_emission_prob(tagsList[s],tList[t])
                                           for s_prime in range(N)])
    bestpathpointer = np.argmax(viterbi[:,-1])
    bestpathprob = max(viterbi[:,-1])
    bestpathtag = []
    for t in range(T-1,-1,-1):
        bestpathtag.insert(0, (tList[t],tagsList[bestpathpointer]))
        bestpathpointer = backpointer[bestpathpointer][t]

    return bestpathtag

In [ ]:
tagsList = [tag for tag in tags]
f = open("./Test_File.txt")
test_data_caps = f.read()
sentences = test_data_caps.split("< EOS >\n")
f = open("testPOS_res.txt","a+")
# for sentence in sentences:
sentence = sentences[2]
print("Tagging....sentence")
result = ""
words = sentence.split("\n")
sentenceTag = words[0]
words_lower = list(filter(lambda x: len(x) != 0,[word.lower() for word in words[1:]]))
tag_res = viterbi(words_lower,tagsList)
polo = zip(words[1:],[wordtag[1] for wordtag in tag_res])
result = sentenceTag + "\n" + "\n".join([w[0]+"/"+w[1] for w in polo])
result = result + "\n< EOS >\n"
print(result)
# f.write(result)
# f.close()

Tagging....sentence
